In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VectorQuantizer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, commitment_cost):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.num_embeddings = num_embeddings
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.uniform_(-1/self.num_embeddings, 1/self.num_embeddings)
        self.commitment_cost = commitment_cost

    def forward(self, inputs):
        flat_inputs = inputs.view(-1, self.embedding_dim)
        distances = (
            torch.sum(flat_inputs**2, dim=1, keepdim=True)
            + torch.sum(self.embedding.weight**2, dim=1)
            - 2 * torch.matmul(flat_inputs, self.embedding.weight.t())
        )
        encoding_indices = torch.argmin(distances, dim=1)
        encodings = F.one_hot(encoding_indices, self.num_embeddings).type(inputs.dtype)
        quantized = torch.matmul(encodings, self.embedding.weight).view_as(inputs)
        e_latent_loss = F.mse_loss(quantized.detach(), inputs)
        q_latent_loss = F.mse_loss(quantized, inputs.detach())
        loss = q_latent_loss + self.commitment_cost * e_latent_loss
        quantized = inputs + (quantized - inputs).detach()
        return quantized, loss, encoding_indices.view(inputs.shape[0], inputs.shape[2], inputs.shape[3])

class VQVAE(nn.Module):
    def __init__(self, in_channels=3, hidden_dim=128, embedding_dim=64, num_embeddings=512, commitment_cost=0.25):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dim, 4, 2, 1),  # 64->32
            nn.ReLU(),
            nn.Conv2d(hidden_dim, hidden_dim, 4, 2, 1),   # 32->16
            nn.ReLU(),
            nn.Conv2d(hidden_dim, embedding_dim, 1)       # kanały = embedding_dim
        )
        self.quantizer = VectorQuantizer(num_embeddings, embedding_dim, commitment_cost)

        self.decoder_convT1 = nn.ConvTranspose2d(embedding_dim, hidden_dim, 4, 2, 1)  # 16->32
        self.decoder_relu1 = nn.ReLU()
        self.decoder_convT2 = nn.ConvTranspose2d(hidden_dim, hidden_dim, 4, 2, 1)     # 32->64
        self.decoder_relu2 = nn.ReLU()
        # Usunięto trzecią warstwę ConvTranspose2d
        self.decoder_conv = nn.Conv2d(hidden_dim, in_channels, 1)
        self.decoder_sigmoid = nn.Sigmoid()

    def forward(self, x):
        z = self.encoder(x)  # (B, embedding_dim, 16, 16)
        quantized, vq_loss, _ = self.quantizer(z)

        x = self.decoder_convT1(quantized)  # 16->32
        x = self.decoder_relu1(x)

        x = self.decoder_convT2(x)           # 32->64
        x = self.decoder_relu2(x)

        x = self.decoder_conv(x)             # (B, 3, 64, 64)
        x = self.decoder_sigmoid(x)

        return x, vq_loss

    def decode(self, quantized):
        x = self.decoder_convT1(quantized)
        x = self.decoder_relu1(x)

        x = self.decoder_convT2(x)
        x = self.decoder_relu2(x)

        x = self.decoder_conv(x)
        x = self.decoder_sigmoid(x)
        return x


    def encode_indices(self, x):
        z = self.encoder(x)
        _, _, indices = self.quantizer(z)
        return indices


In [13]:
import torch
import torch.nn as nn

class PixelCNN(nn.Module):
    def __init__(self, num_embeddings, image_size, n_channels=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, n_channels, 7, padding=3),
            nn.ReLU(),
            *[nn.Sequential(nn.Conv2d(n_channels, n_channels, 3, padding=1), nn.ReLU()) for _ in range(6)],
            nn.Conv2d(n_channels, num_embeddings, 1)
        )
        self.image_size = image_size

    def forward(self, x):
        return self.net(x)

    def sample(self, device, num_samples):
        samples = torch.zeros((num_samples, 1, self.image_size, self.image_size), dtype=torch.long).to(device)
        for i in range(self.image_size):
            for j in range(self.image_size):
                logits = self.forward(samples.float())
                probs = torch.softmax(logits[:, :, i, j], dim=1)
                samples[:, 0, i, j] = torch.multinomial(probs, 1).squeeze(-1)
        return samples


In [14]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision.utils import save_image
import os
from tqdm import tqdm
epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.ToTensor()
])
dataset = ImageFolder("data", transform=transform)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

vqvae = VQVAE().to(device)
optimizer = torch.optim.Adam(vqvae.parameters(), lr=2e-4)

for epoch in range(epochs):
    total_loss = 0
    with tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}") as pbar:
        for x, _ in pbar:
            x = x.to(device)
            x_recon, vq_loss = vqvae(x)
            recon_loss = torch.nn.functional.mse_loss(x_recon, x)
            loss = recon_loss + vq_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
    print(f"Epoch {epoch+1}: avg loss={total_loss / len(loader):.4f}")



Epoch 1/50: 100%|██████████| 467/467 [00:16<00:00, 28.21it/s, loss=0.345]


Epoch 1: avg loss=0.4464


Epoch 2/50: 100%|██████████| 467/467 [00:16<00:00, 28.92it/s, loss=0.246]


Epoch 2: avg loss=0.2853


Epoch 3/50: 100%|██████████| 467/467 [00:16<00:00, 28.86it/s, loss=0.166]


...

Epoch 48: avg loss=0.0810


Epoch 49/50: 100%|██████████| 467/467 [00:16<00:00, 28.58it/s, loss=0.0792]


Epoch 49: avg loss=0.0807


Epoch 50/50: 100%|██████████| 467/467 [00:16<00:00, 28.94it/s, loss=0.0739]

Epoch 50: avg loss=0.0803


In [15]:

# Zapis kodów latentnych
codes = []
print("Encoding latent indices...")
with torch.no_grad():
    for x, _ in tqdm(loader, desc="Encoding"):
        x = x.to(device)
        idx = vqvae.encode_indices(x)
        codes.append(idx)
codes = torch.cat(codes)
# Trening PixelCNN na kodach
pixelcnn = PixelCNN(num_embeddings=512, image_size=codes.shape[1]).to(device)
optimizer_pcnn = torch.optim.Adam(pixelcnn.parameters(), lr=2e-4)

codes = codes.unsqueeze(1).float()  # [B, 1, H, W]

print("Training PixelCNN...")
for epoch in range(epochs):
    total_loss = 0
    with tqdm(range(0, codes.size(0), 64), desc=f"PixelCNN Epoch {epoch+1}/{epochs}") as pbar:
        for i in pbar:
            batch = codes[i:i+64].to(device)
            logits = pixelcnn(batch)
            loss = torch.nn.functional.cross_entropy(logits, batch.squeeze(1).long())
            optimizer_pcnn.zero_grad()
            loss.backward()
            optimizer_pcnn.step()
            total_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
    print(f"[PixelCNN] Epoch {epoch+1}: avg loss={total_loss / (codes.size(0)//64):.4f}")


Encoding latent indices...


Encoding: 100%|██████████| 467/467 [00:09<00:00, 49.58it/s]


Training PixelCNN...


PixelCNN Epoch 1/50: 100%|██████████| 467/467 [00:01<00:00, 248.37it/s, loss=3.88]


[PixelCNN] Epoch 1: avg loss=4.9095


...

[PixelCNN] Epoch 48: avg loss=2.4853


PixelCNN Epoch 49/50: 100%|██████████| 467/467 [00:01<00:00, 269.76it/s, loss=2.26]


[PixelCNN] Epoch 49: avg loss=2.4802


PixelCNN Epoch 50/50: 100%|██████████| 467/467 [00:01<00:00, 271.53it/s, loss=2.27]

[PixelCNN] Epoch 50: avg loss=2.4742


In [19]:
samples = pixelcnn.sample(device, 64)  # [64, 1, H, W] lub [64, H, W]
samples = samples.squeeze(1).long()    # [64, H, W]

# Zamiana indeksów na embeddingi: [B, H, W] -> [B, H, W, C]
embeddings = vqvae.quantizer.embedding(samples)  

# Zamiana na [B, C, H, W]
embeddings = embeddings.permute(0, 3, 1, 2).contiguous()

# Dekodowanie
with torch.no_grad():
    recon = vqvae.decode(embeddings)   # uzywając metody decode z klasy VQVAE

# Zapis
save_image(recon, "generated/generated_cats_lr_2e-04.png", nrow=8)

In [23]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision.utils import save_image
import os
from tqdm import tqdm
epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.ToTensor()
])
dataset = ImageFolder("data", transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

vqvae = VQVAE().to(device)
optimizer = torch.optim.Adam(vqvae.parameters(), lr=2e-4)

for epoch in range(epochs):
    total_loss = 0
    with tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}") as pbar:
        for x, _ in pbar:
            x = x.to(device)
            x_recon, vq_loss = vqvae(x)
            recon_loss = torch.nn.functional.mse_loss(x_recon, x)
            loss = recon_loss + vq_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
    print(f"Epoch {epoch+1}: avg loss={total_loss / len(loader):.4f}")



Epoch 1/50: 100%|██████████| 933/933 [00:17<00:00, 53.74it/s, loss=0.333]


Epoch 1: avg loss=0.4338


Epoch 2/50: 100%|██████████| 933/933 [00:17<00:00, 53.92it/s, loss=0.177]


Epoch 2: avg loss=0.2128


Epoch 3/50: 100%|██████████| 933/933 [00:17<00:00, 54.31it/s, loss=0.15] 


...

Epoch 48: avg loss=0.0783


Epoch 49/50: 100%|██████████| 933/933 [00:17<00:00, 53.01it/s, loss=0.0756]


Epoch 49: avg loss=0.0781


Epoch 50/50: 100%|██████████| 933/933 [00:17<00:00, 54.18it/s, loss=0.0762]

Epoch 50: avg loss=0.0778


In [24]:

# Zapis kodów latentnych
codes = []
print("Encoding latent indices...")
with torch.no_grad():
    for x, _ in tqdm(loader, desc="Encoding"):
        x = x.to(device)
        idx = vqvae.encode_indices(x)
        codes.append(idx)
codes = torch.cat(codes)
# Trening PixelCNN na kodach
pixelcnn = PixelCNN(num_embeddings=512, image_size=codes.shape[1]).to(device)
optimizer_pcnn = torch.optim.Adam(pixelcnn.parameters(), lr=2e-3)

codes = codes.unsqueeze(1).float()  # [B, 1, H, W]

print("Training PixelCNN...")
for epoch in range(epochs):
    total_loss = 0
    with tqdm(range(0, codes.size(0), 64), desc=f"PixelCNN Epoch {epoch+1}/{epochs}") as pbar:
        for i in pbar:
            batch = codes[i:i+64].to(device)
            logits = pixelcnn(batch)
            loss = torch.nn.functional.cross_entropy(logits, batch.squeeze(1).long())
            optimizer_pcnn.zero_grad()
            loss.backward()
            optimizer_pcnn.step()
            total_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
    print(f"[PixelCNN] Epoch {epoch+1}: avg loss={total_loss / (codes.size(0)//64):.4f}")


Encoding latent indices...


Encoding: 100%|██████████| 933/933 [00:09<00:00, 93.71it/s]


Training PixelCNN...


PixelCNN Epoch 1/50: 100%|██████████| 467/467 [00:01<00:00, 235.81it/s, loss=5.25]


[PixelCNN] Epoch 1: avg loss=5.5023


...

[PixelCNN] Epoch 48: avg loss=2.4184


PixelCNN Epoch 49/50: 100%|██████████| 467/467 [00:01<00:00, 250.34it/s, loss=2.38]


[PixelCNN] Epoch 49: avg loss=2.5703


PixelCNN Epoch 50/50: 100%|██████████| 467/467 [00:01<00:00, 255.39it/s, loss=2.43]

[PixelCNN] Epoch 50: avg loss=2.4175


In [26]:
samples = pixelcnn.sample(device, 64)  # [64, 1, H, W] lub [64, H, W]
samples = samples.squeeze(1).long()    # [64, H, W]

# Zamiana indeksów na embeddingi: [B, H, W] -> [B, H, W, C]
embeddings = vqvae.quantizer.embedding(samples)  

# Zamiana na [B, C, H, W]
embeddings = embeddings.permute(0, 3, 1, 2).contiguous()

# Dekodowanie
with torch.no_grad():
    recon = vqvae.decode(embeddings)   # uzywając metody decode z klasy VQVAE


os.makedirs("generated", exist_ok=True)
save_image(recon, "generated/generated_cats_lr_2e-03.png", nrow=8)